In [22]:
import pandas as pd
import numpy as np
import ast

df_fraud = pd.read_csv('data/df_fraud_embedding.csv', index_col=False)
df_injury = pd.read_csv('data/df_fraud_embedding.csv', index_col=False)
df_labor = pd.read_csv('data/df_fraud_embedding.csv', index_col=False)

In [23]:
import pandas as pd
import numpy as np
import ast

df_fraud['판례내용이유임베딩'] = df_fraud['판례내용이유임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_fraud['키워드임베딩'] = df_fraud['키워드임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_fraud['판결요지임베딩'] = df_fraud['판결요지임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_fraud['판시사항임베딩'] = df_fraud['판시사항임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)

In [24]:
df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4178 entries, 0 to 4177
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4178 non-null   int64 
 1   사건명            4176 non-null   object
 2   사건번호           4178 non-null   object
 3   선고일자           4178 non-null   object
 4   법원명            4178 non-null   object
 5   사건종류명          4178 non-null   object
 6   판례상세링크         4178 non-null   object
 7   판례정보일련번호       4178 non-null   int64 
 8   선고             4178 non-null   object
 9   판결유형           4178 non-null   object
 10  판시사항           3025 non-null   object
 11  판결요지           2479 non-null   object
 12  참조조문           2998 non-null   object
 13  참조판례           1805 non-null   object
 14  판례내용           4178 non-null   object
 15  판례내용_상단        4178 non-null   object
 16  판례내용_이유        4178 non-null   object
 17  판례내용_이유_전처리    4178 non-null   object
 18  판례내용_이유_불용어제거  4178 non-null

In [26]:
df_fraud['키워드'].iloc[0]

"['지급', '계약', '사기죄', '채권', '신청', '회사', '공사', '의사', '오해', '거래']"

In [ ]:
df_fraud['키워드'] = df_fraud['키워드'].apply(lambda x: x.strip('[]').replace("'","").replace(' ','').split(','))
df_fraud['키워드'].head()

In [34]:
df_fraud['키워드'].iloc[0]

['지급', ' 계약', ' 사기죄', ' 채권', ' 신청', ' 회사', ' 공사', ' 의사', ' 오해', ' 거래']

# 유사도 계산

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 키워드에 대한 임베딩을 찾기위한 함수
def get_keyword_embedding(user_keyword, df):
    # 키워드가 포함된 행을 필터링
    matching_rows = df_fraud[df_fraud['키워드'].apply(lambda x: user_keyword in x if isinstance(x, list) else False)]
    
    if not matching_rows.empty:
        # 첫번째 매칭된 키워드의 임베딩 반환
        return matching_rows['키워드임베딩'].iloc[0]
    else:
        return None
    

user_keyword = '지급'
user_keyword_embedding = get_keyword_embedding(user_keyword, df_fraud)

if user_keyword_embedding is not None:
    # 판례내용 임베딩 유사도 계산 및 사건명 추출
    content_similarity_scores = []
    
    for idx, row in df_fraud.iterrows():
        if row['판례내용이유임베딩'] is not None and not np.isnan(row['판례내용이유임베딩']).any():
            content_similarity = cosine_similarity(user_keyword_embedding.reshape(1,-1), row['판례내용이유임베딩'].reshape(1,-1)).flatten()[0]
            content_similarity_scores.append((row['사건명'], content_similarity))
    # 유사도 기준으로 정렬 및 상위 10개 선택
    sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
    top_n_contents = sorted_content_scores[:10]
    
    # 추천 사건명 출력
    print('사용자 키워드와 관련된 사건명:')
    for 사건번호, score in top_n_contents:
        사건명 = df_fraud.loc[df_fraud['사건번호']==사건번호, '사건명'].values
        사건명 = 사건명[0] if 사건명.size > 0 else '사건명 없음'
        print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")
else:
    print('해당 키워드에 대한 임베딩이 없습니다.')


사용자 키워드와 관련된 사건명:
사건번호: 보호감호,상습사기, 사건명: 사건명 없음, 유사도: 0.7024
사건번호: 출입국관리법위반, 사건명: 사건명 없음, 유사도: 0.6992
사건번호: 부당이득금, 사건명: 사건명 없음, 유사도: 0.6934
사건번호: 손해배상(기), 사건명: 사건명 없음, 유사도: 0.6780
사건번호: 특정범죄가중처벌등에관한법률위반(조세), 사건명: 사건명 없음, 유사도: 0.6761
사건번호: 법인세부과처분취소청구사건, 사건명: 사건명 없음, 유사도: 0.6728
사건번호: 보증채무금, 사건명: 사건명 없음, 유사도: 0.6718
사건번호: 보호감호ㆍ상습사기, 사건명: 사건명 없음, 유사도: 0.6716
사건번호: 수입신용장결제대금지급, 사건명: 사건명 없음, 유사도: 0.6702
사건번호: 사기미수·사문서변조(인정된 죄명 : 공문서변조)·변조사문서행사 (인정된 죄명 : 변        조공문서행사), 사건명: 사건명 없음, 유사도: 0.6692


In [35]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 사용자 키워드에 대한 임베딩을 찾기위한 함수
def get_keyword_embedding(user_keyword, df):
    # 키워드가 포함된 행을 필터링
    matching_rows = df[df['키워드'].apply(lambda x: user_keyword in x if isinstance(x, list) else False)]
    
    if not matching_rows.empty:
        # 첫번째 매칭된 키워드의 임베딩 반환
        return matching_rows['키워드임베딩'].iloc[0]
    else:
        return None
    

user_keyword = '지급'
user_keyword_embedding = get_keyword_embedding(user_keyword, df_fraud)

if user_keyword_embedding is not None:
    # 판례내용 임베딩 유사도 계산 및 사건번호 추출
    content_similarity_scores = []
    
    for idx, row in df_fraud.iterrows():
        if row['판례내용이유임베딩'] is not None and not np.isnan(row['판례내용이유임베딩']).any():
            content_similarity = cosine_similarity(user_keyword_embedding.reshape(1, -1), row['판례내용이유임베딩'].reshape(1, -1)).flatten()[0]
            content_similarity_scores.append((row['사건번호'], content_similarity))
    
    # 유사도 기준으로 정렬 및 상위 10개 선택
    sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
    top_n_contents = sorted_content_scores[:10]
    
    # 추천 사건명 출력
    print('사용자 키워드와 관련된 사건명:')
    for 사건번호, score in top_n_contents:
        사건명 = df_fraud.loc[df_fraud['사건번호'] == 사건번호, '사건명'].values
        사건명 = 사건명[0] if 사건명.size > 0 else '사건명 없음'
        print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")
else:
    print('해당 키워드에 대한 임베딩이 없습니다.')


사용자 키워드와 관련된 사건명:
사건번호: 86감도48, 사건명: 보호감호,상습사기, 유사도: 0.7024
사건번호: 2009노4971, 사건명: 출입국관리법위반, 유사도: 0.6992
사건번호: 2003가합1087, 사건명: 부당이득금, 유사도: 0.6934
사건번호: 2001가합1092, 사건명: 손해배상(기), 유사도: 0.6780
사건번호: 2012고합464, 사건명: 특정범죄가중처벌등에관한법률위반(조세), 유사도: 0.6761
사건번호: 82구238, 사건명: 법인세부과처분취소청구사건, 유사도: 0.6728
사건번호: 2004가합1589, 사건명: 보증채무금, 유사도: 0.6718
사건번호: 85감도170, 사건명: 보호감호ㆍ상습사기, 유사도: 0.6716
사건번호: 92가합19434, 사건명: 수입신용장결제대금지급, 유사도: 0.6702
사건번호: 2003도2144, 사건명: 사기미수·사문서변조(인정된 죄명 : 공문서변조)·변조사문서행사 (인정된 죄명 : 변        조공문서행사), 유사도: 0.6692
